# 감정 분석

<br>

- 파이프라인이 처리하는 어떤것이든 각 토큰에는 어떠한 정보가 담겨있음.
- 단어의 감정(sentiment), 그 단어가 불러 일으키는 전반적인 느낌은 그런 정보의 중요한 일부분.
- 감정분석은 NLP의 일반적인 응용 중 하나.
    - 이를 응용해 스팸 필터링, 챗봇응용 등이 있음.

<br>

- 파이프라인이 텍스트 조각에 담긴 감정을 분석해 수치화하려면 어떻게 해야 할까. 크게 2가지 접근방식.
    1. 사람이 직접 작성한 **규칙 기반 알고리즘** 사용 $\rightarrow$ **발견법(heuristics)에 기초한 감성 분석**
        - 텍스트에서 **특정 키워드**를 찾고, **키워드 들에 부여된 수치 점수를 취합**.
        - **특정 단어와 점수 쌍을 담은 Dict 자료구조**가 필요. 사람이 직접 만들어야 함.
        - 주로 **VADER**를 이용.
    2. 컴퓨터가 자료로부터 직접 배우는 **ML모형** 사용
        - **분류명이 붙은** 문장/문서집합 을 통해 ML모형 훈련, 규칙 생성.
        - ML모형은 입력 텍스트에 대한 수치적 감정 점수를 출력하도록 훈련.
        - **'정답'에 해당하는 감정 점수가 부여된 텍스트 조각들로 이뤄진 대량의 자료가 필요**한 지도학습 모형.
            - 주로 해시태그가 많이 붙는 tweet 데이터가 많이 쓰임.
            - 또는 의견과 함께 별점 평가를 받아 활용할 수도 있음.
        - **Naive-Bayes**라 불리는 토큰 기반 ML 알고리즘을 주로 이용.

## VADER : 규칙기반 감정분석기

<br>

- **V**alence **A**ware **D**ictionary for s**E**ntiment **R**easoning (감정 추론을 위한 결합가 인식 사전)의 약자.
- NLTK의 경우 nltk.sentiment.vader 를 통해 사용 가능.
- vaderSentiment 라는 패키지도 있음. 예제는 이걸 쓸것.

        pip install vaderSentiment

In [1]:
!pip install vaderSentiment

You should consider upgrading via the 'c:\users\skdbs\anaconda3\python.exe -m pip install --upgrade pip' command.


In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sa = SentimentIntensityAnalyzer()
sa.lexicon # lexicon에 토큰-감정 점수 쌍들이 들어있음.

# VADER는 이모티콘 사용. 분석이 잘 되려면 토큰 생성기가 문장 부호들을 제거하지 않는것이 바람직.
sa.polarity_scores(text={
    ':(' : -1.9,
    ':)' : 2.0,
    'pls' : 0.3,
    'plz' : 0.3,
    'great' : 3.1
})
[(tok, score) for tok, score in sa.lexicon.items() if " " in tok]

[("( '}{' )", 1.6),
 ("can't stand", -2.0),
 ('fed up', -1.8),
 ('screwed up', -1.5)]

In [5]:
sa.polarity_scores(text="Python is very readable and it's great for NLP.")

{'neg': 0.0, 'neu': 0.661, 'pos': 0.339, 'compound': 0.6249}

    VADER 알고리즘음 감정의 세기(intensity)를 긍정(pos), 부정(neg), 중립(neu)로 분류.
    이 3개의 점수를 담은 하나의 복합 자료 구조를 return 함.

In [6]:
sa.polarity_scores(text="Python is not a bad choice for most applications.")

{'neg': 0.0, 'neu': 0.737, 'pos': 0.263, 'compound': 0.431}

    not 같은 부정어를 상당히 잘 처리함.
    not bad의 긍정 점수는 great보다 약간만 낮을 정도로 높은 편.
    VADER의 내장 토큰 생성기는 자신의 어휘집엔 없는 단어들을 모두 무시, n-gram은 전혀 고려 X.

In [7]:
corpus = ["Absolutely perfect!! Love it! :-) :-) :-)",
          "Horrible! Completely useless :(",
          "It was OK. Some good and some bad things."]
for doc in corpus:
    scores = sa.polarity_scores(doc)
    print('{:+}: {}'.format(scores['compound'], doc))

+0.9455: Absolutely perfect!! Love it! :-) :-) :-)
-0.8768: Horrible! Completely useless :(
-0.1531: It was OK. Some good and some bad things.


    상당히 잘 동작하고 있음.

- VADER의 유일한 단점은 모든 단어가 아니라 약 7500개의 단어만을 고려한다는 것.
- 모든 단어로 감정 점수를 측정해야 한다면 할 일이 엄청나게 늘어남.
    - 일일이 점수를 매기거나, lexicon 사전 자료 구조에 수많은 커스텀 단어를 추가해야 함.
- 규칙 기반 접근방식은 대상 자연어를 이해하지 못하면 거의 불가능한 일.
- 그래서 기계학습에 기초한 감정 분석기를 사용하는 것이 편리하다.

## Naive-Bayes Model

<br>

